In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocess the data
## Drop irrelevant columns
data = data.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [4]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## Encode Categorical variable

label_encoder_gender = LabelEncoder()
data["Gender"]=label_encoder_gender.fit_transform(data["Gender"])

In [6]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [9]:
## Onehot encode geography column

from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[["Geography"]])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [15]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [12]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [18]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [19]:
#combine all the columns ie, onehot encoded columns with original data
data = pd.concat([data.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [20]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [21]:
# save the file into pixel file encoders and scalar

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_gro.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo, file)
    

In [22]:
## split dataset into independent and dependent features

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [23]:
## split dataset into independent and dependent features
X = data.drop('Exited',axis=1)
y = data['Exited']

# Split the dataset in training and testing sets
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state=42)

# Scale these features
scaler= StandardScaler()

X_train=scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [25]:
with open('scalar.pkl','wb') as file:
    pickle.dump(scaler,file)

In [26]:
# ANN implementation

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [31]:
X_train.shape[1],

(12,)

In [32]:
# Build ANN model

model = Sequential([
    Dense(64,activation='relu',input_shape = (X_train.shape[1],)), # First hidden layer connected with Input layer
    Dense(32,activation='relu'), # Hidden layer 2
    Dense(1,activation='sigmoid') # Output layer
])

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
import tensorflow
opt =tensorflow.keras.optimizers.Adam(learning_rate=0.01) # if you want to give dynamic learning rate
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [38]:
# Compile model
model.compile(optimizer=opt, loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
# Setup the tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir="logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir = log_dir, histogram_freq=1)

In [44]:
# Setup Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True) 

In [45]:
# Training the model
history=model.fit(
    X_train, y_train, validation_data=(X_test,y_test), epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3333 - accuracy: 0.8641 - val_loss: 0.3493 - val_accuracy: 0.8555
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3335 - accuracy: 0.8633 - val_loss: 0.3558 - val_accuracy: 0.8590
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3302 - accuracy: 0.8648 - val_loss: 0.3483 - val_accuracy: 0.8590
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3280 - accuracy: 0.8636 - val_loss: 0.3504 - val_accuracy: 0.8645
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3278 - accuracy: 0.8634 - val_loss: 0.3476 - val_accuracy: 0.8595
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3233 - accuracy: 0.8662 - val_loss: 0.3613 - val_accuracy: 0.8585
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3200 - accuracy: 0.8689 - val_loss: 0.3565 - val_accuracy: 0.8530

In [46]:
model.save('model.h5') # h5 as it will be compatible with keras

d:\Projects\churn_rate_ann\m\myenvname\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [48]:
# load tensorboard extension
%load_ext tensorboard

In [54]:
%tensorboard --logdir logs/fit20251117-203125

Reusing TensorBoard on port 6008 (pid 31704), started 0:00:11 ago. (Use '!kill 31704' to kill it.)